# Problem Statement
Your assignment today is to write a Sim2L that solves a pair of ordinary differential equations describing the motion of a simple pedulum with damping applied. 
![Simple Pedulum with Damping](pendulumDiagram.gif)
The above diagram indicates the forces acting on a pendulum swinging from a fixed point. The equation describing the motion is:
<br>
<br>
$$\frac {d^{2}\theta} {dt^{2}} = - \frac {b}{m} \frac {d \theta} {dt} - \frac {g} {L} sin \theta $$
where:
<br>
$ \theta $ - angle measured from vertical (radian)
<br>
$ t $ - time (time)
<br>
$ b $ - damping coeffient (mass-radian/time)
<br>
$ m $ - pedulum mass (mass)
<br>
$ g $ - gravitational constant (mass/time$^{2}$)
<br>
$ L $ - pendulum length (length)
<br>
<br>
The second order differential equation can be broken down into a pair of first order differential more suitable for numerical solution.
<br>
<br>
$$ \frac {d \theta} {dt} = v $$
<br>
$$ \frac {dv} {dt} = - \frac {g} {L} sin \theta - \frac {b}{m} v $$
where:
<br>
$ v $ - angular velocity (radian/time)
<br>
<br>
It is suggested that the *solve_ivp* method from scipy be used to compute the numerical solution.  Details of the *solve_ivp* method can be found online.  The Sim2L results should provide enough information to plot position and velocity as functions of time.

In [ ]:
DESCRIPTION = """Pedulum Motion with damping SimTool"""

In [ ]:
EXTRA_FILES = []

In [ ]:
%load_ext yamlmagic

# Input Conditions and Parameters
Based on the pedulum model described above and your choice of numerical method, choose a suitable set of conditions and parameters for the Sim2L. 

In [ ]:
%%yaml INPUTS

length:
    type: Number
    description: Pendulum length
    value: 1
    units: m
    min: 0.1
    max: 100
mass:
    type: Number
    description: Pendulum mass
    value: 1
    units: kg
    min: 0.001
damp:
    type: Number
    description: Damping coefficient
    value: 0.1
    units: kg rad/s
    min: 0.0
initialPosition:
    type: Number
    description: Initial location
    value: 0.
    units: rad
initialVelocity:
    type: Number
    description: Initial velocity
    value: 1.
    units: rad/s
    min: -100
    max: +100
timeHorizon:
    type: Number
    description: Simulation duration
    units: s
    value: 100.
    min: 0
    max: 1000
nTimePoints:
    type: Integer
    description: Number of points to record simulation results
    value: 100
    min: 2

In [ ]:
from simtool import getValidatedInputs

defaultInputs = getValidatedInputs(INPUTS)
if defaultInputs:
    globals().update(defaultInputs)

# Output Results
Based on the pedulum model described above and your choice of numerical method, choose a suitable set of computed results for the Sim2L. 

In [ ]:
%%yaml OUTPUTS

success:
    type: Boolean
    description: Was simulation successful
terminationMessage:
    type: Text
    description: Solver termination message
time:
    type: Array
    description: Times position recorded
    units: s
position:
    type: Array
    description: Positions at time values
    units: rad
velocity:
    type: Array
    description: Velocity at time values
    units: rad/s

# Function for computing position and velocity derivatives
Based on *solve_ivp* method requirements write a function that computes variables derivates for a given set of variable and parameter values.

In [ ]:
from math import sin
from scipy.integrate import solve_ivp
import numpy as np

def model(time,variables,damp,length,mass):
    position = variables[0]
    velocity = variables[1]
    gConstant = 9.81 # m/s2
    
    dPosition_dt = velocity
    dVelocity_dt = -(damp/mass)*velocity - (gConstant/length)*sin(position)
    
    return [dPosition_dt,dVelocity_dt]

# Initial Conditions
Set variable initial conditions according to *solve_ivp* requirements.  

In [ ]:
initialVariables = [initialPosition,initialVelocity]
timePoints = np.linspace(0.,timeHorizon,nTimePoints)

# Solve Equations
The *solve_ivp* is capable of solving problems more complex or difficult than what is needed for the pendulum problem.  Here is a simplified version sufficient for the problem at hand.

**result = scipy.integrate.solve_ivp(fun, t_span, y0, t_eval=fixedTimePoints, args=modelParameters)**
<br>
<br>
**fun**: callable function that computes dy/dt for given t and y.
<br>
**t_span**: Interval of integration (t0,tf)
<br>
**y0**: Initial y values at t0.
<br>
**t_eval**: Optional set of fixed time points at which to record y values
<br>
**modelParameters** = Tuple of fixed model parameters
<br>
<br>
It is also not necessary to consider all that *solve_ivp* is capable of returning.  The return values of interest are:
<br>
<br>
**y1** = result['y'][0,:]
<br>
**y2** = result['y'][1,:]
<br>
**success** = result['success']
<br>
**message** = result['message']
<br>
**t** = result['t']
<br>

In [ ]:
result = solve_ivp(model,(0.,timeHorizon),initialVariables,t_eval=timePoints,args=(damp,length,mass))

# Extract results
*solve_ivp* computes many things including position and velocity trajectories.  Extract the results you need to assign values to all Sim2L outputs.

In [ ]:
success = result['success']
terminationMessage = result['message']
time = result['t']
position = result['y'][0,:]
velocity = result['y'][1,:]

# Save results

In [ ]:
from simtool import DB

db = DB(OUTPUTS)

db.save('success', success)
db.save('terminationMessage', terminationMessage)
db.save('time', time)
db.save('position', position)
db.save('velocity', velocity)